# Regression Exercise 

California Housing Data

This data set contains information about all the block groups in California from the 1990 Census. In this sample a block group on average includes 1425.5 individuals living in a geographically compact area. 

The task is to aproximate the median house value of each block from the values of the rest of the variables. 

 It has been obtained from the LIACC repository. The original page where the data set can be found is: http://www.liaad.up.pt/~ltorgo/Regression/DataSets.html.
 

The Features:
 
* housingMedianAge: continuous. 
* totalRooms: continuous. 
* totalBedrooms: continuous. 
* population: continuous. 
* households: continuous. 
* medianIncome: continuous. 
* medianHouseValue: continuous. 

## The Data

** Import the cal_housing_clean.csv file with pandas. Separate it into a training (70%) and testing set(30%).**

In [70]:
import pandas as pd

In [71]:
house = pd.read_csv("/home/priyank/Downloads/FULL_TENSORFLOW_NOTES__AND_DATA/Tensorflow-Bootcamp-master/02-TensorFlow-Basics/cal_housing_clean.csv")

In [72]:
house.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
0,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


In [73]:
x_data = house.drop('medianHouseValue',axis=1)

In [74]:
x_data.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome
0,41.0,880.0,129.0,322.0,126.0,8.3252
1,21.0,7099.0,1106.0,2401.0,1138.0,8.3014
2,52.0,1467.0,190.0,496.0,177.0,7.2574
3,52.0,1274.0,235.0,558.0,219.0,5.6431
4,52.0,1627.0,280.0,565.0,259.0,3.8462


In [75]:
labels = house['medianHouseValue']

In [76]:
labels.head()

0    452600.0
1    358500.0
2    352100.0
3    341300.0
4    342200.0
Name: medianHouseValue, dtype: float64

In [77]:
from sklearn.model_selection import train_test_split

In [78]:
X_train, X_test, y_train, y_test = train_test_split(
...     x_data, labels, test_size=0.30, random_state=101)

### Scale the Feature Data

** Use sklearn preprocessing to create a MinMaxScaler for the feature data. Fit this scaler only to the training data. Then use it to transform X_test and X_train. Then use the scaled X_test and X_train along with pd.Dataframe to re-create two dataframes of scaled data.**

In [79]:
from sklearn.preprocessing import MinMaxScaler

In [80]:
scalar_model = MinMaxScaler()

In [81]:
scalar_model.fit(X_train)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [82]:
X_train_tsfd = scalar_model.transform(X_train)

In [83]:
X_test_tsfd = scalar_model.transform(X_test)

In [84]:
type(X_train_tsfd)

numpy.ndarray

In [85]:
X_train_df = pd.DataFrame(data = X_train_tsfd, columns = ["housingMedianAge","totalRooms","totalBedrooms","population","households","medianIncome"])

In [86]:
X_train_df.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome
0,0.352941,0.069688,0.117163,0.048769,0.115442,0.142508
1,0.607843,0.011242,0.015673,0.008367,0.014142,0.045027
2,0.666667,0.025230,0.031347,0.020971,0.030258,0.212866
3,0.666667,0.032530,0.033830,0.024752,0.030094,0.298651
4,0.294118,0.031919,0.035692,0.019466,0.034863,0.272631


In [87]:
X_test_df = pd.DataFrame(data = X_test_tsfd, columns = ["housingMedianAge","totalRooms","totalBedrooms","population","households","medianIncome"])

In [88]:
X_test_df.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome
0,0.686275,0.046264,0.045158,0.025873,0.048841,0.353133
1,0.705882,0.027417,0.020795,0.012709,0.023187,0.770182
2,0.901961,0.032326,0.040813,0.041662,0.042592,0.133626
3,0.313725,0.043212,0.046089,0.032840,0.048018,0.263576
4,0.411765,0.088433,0.069367,0.043728,0.072192,0.660046


### Create Feature Columns

** Create the necessary tf.feature_column objects for the estimator. They should all be trated as continuous numeric_columns. **

In [89]:
house.columns

Index(['housingMedianAge', 'totalRooms', 'totalBedrooms', 'population',
       'households', 'medianIncome', 'medianHouseValue'],
      dtype='object')

In [90]:
import tensorflow as tf

In [91]:
housingMedianAge_ = tf.feature_column.numeric_column("housingMedianAge")
totalRooms_ = tf.feature_column.numeric_column("totalRooms")
totalBedrooms_ = tf.feature_column.numeric_column("totalBedrooms")
population_ = tf.feature_column.numeric_column("population")
households_ = tf.feature_column.numeric_column("households")
medianIncome_ = tf.feature_column.numeric_column("medianIncome")
medianHouseValue_ = tf.feature_column.numeric_column("medianHouseValue")

In [92]:
feat_cols = [housingMedianAge_, totalRooms_, totalBedrooms_, population_, households_, medianIncome_]

** Create the input function for the estimator object. (play around with batch_size and num_epochs)**

In [93]:
input_func = tf.estimator.inputs.pandas_input_fn(x = X_train, y = y_train, 
                                                batch_size = 10, num_epochs = 1000,
                                                shuffle = True)

** Create the estimator model. Use a DNNRegressor. Play around with the hidden units! **

In [94]:
dnn_model = tf.estimator.DNNRegressor(hidden_units=[6,6,6],feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_tf_random_seed': 1, '_session_config': None, '_save_checkpoints_steps': None, '_model_dir': '/tmp/tmpdo3byo3a', '_save_summary_steps': 100, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100}


##### ** Train the model for ~1,000 steps. (Later come back to this and train it for more and check for improvement) **

In [114]:
dnn_model.train(input_fn= input_func ,steps= 25000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from /tmp/tmpdo3byo3a/model.ckpt-1000
INFO:tensorflow:Saving checkpoints for 1001 into /tmp/tmpdo3byo3a/model.ckpt.
INFO:tensorflow:loss = 190117860000.0, step = 1001
INFO:tensorflow:global_step/sec: 325.86
INFO:tensorflow:loss = 1325895000000.0, step = 1101 (0.312 sec)
INFO:tensorflow:global_step/sec: 366.43
INFO:tensorflow:loss = 328709830000.0, step = 1201 (0.273 sec)
INFO:tensorflow:global_step/sec: 352.547
INFO:tensorflow:loss = 150772700000.0, step = 1301 (0.283 sec)
INFO:tensorflow:global_step/sec: 341.636
INFO:tensorflow:loss = 91190250000.0, step = 1401 (0.297 sec)
INFO:tensorflow:global_step/sec: 314.934
INFO:tensorflow:loss = 239332970000.0, step = 1501 (0.312 sec)
INFO:tensorflow:global_step/sec: 254.075
INFO:tensorflow:loss = 197580050000.0, step = 1601 (0.394 sec)
INFO:tensorflow:global_step/sec: 322.959
INFO:tensorflow:loss = 415014780000.0, step = 1701 (0.309 sec)
INFO:tensorflow:global_ste

INFO:tensorflow:global_step/sec: 355.776
INFO:tensorflow:loss = 232704620000.0, step = 8801 (0.285 sec)
INFO:tensorflow:global_step/sec: 349.435
INFO:tensorflow:loss = 400780820000.0, step = 8901 (0.280 sec)
INFO:tensorflow:global_step/sec: 361.144
INFO:tensorflow:loss = 123593330000.0, step = 9001 (0.288 sec)
INFO:tensorflow:global_step/sec: 298.502
INFO:tensorflow:loss = 126433790000.0, step = 9101 (0.328 sec)
INFO:tensorflow:global_step/sec: 349.527
INFO:tensorflow:loss = 97042150000.0, step = 9201 (0.283 sec)
INFO:tensorflow:global_step/sec: 270.098
INFO:tensorflow:loss = 91648150000.0, step = 9301 (0.372 sec)
INFO:tensorflow:global_step/sec: 314.028
INFO:tensorflow:loss = 110463600000.0, step = 9401 (0.319 sec)
INFO:tensorflow:global_step/sec: 345.51
INFO:tensorflow:loss = 192396210000.0, step = 9501 (0.289 sec)
INFO:tensorflow:global_step/sec: 379.854
INFO:tensorflow:loss = 435111760000.0, step = 9601 (0.266 sec)
INFO:tensorflow:global_step/sec: 339.175
INFO:tensorflow:loss = 144

INFO:tensorflow:loss = 185853530000.0, step = 16601 (0.277 sec)
INFO:tensorflow:global_step/sec: 352.073
INFO:tensorflow:loss = 142540750000.0, step = 16701 (0.282 sec)
INFO:tensorflow:global_step/sec: 355.982
INFO:tensorflow:loss = 220594200000.0, step = 16801 (0.280 sec)
INFO:tensorflow:global_step/sec: 383.178
INFO:tensorflow:loss = 245487550000.0, step = 16901 (0.264 sec)
INFO:tensorflow:global_step/sec: 329.736
INFO:tensorflow:loss = 125734670000.0, step = 17001 (0.304 sec)
INFO:tensorflow:global_step/sec: 364.798
INFO:tensorflow:loss = 52645960000.0, step = 17101 (0.273 sec)
INFO:tensorflow:global_step/sec: 366.931
INFO:tensorflow:loss = 485366240000.0, step = 17201 (0.273 sec)
INFO:tensorflow:global_step/sec: 363.454
INFO:tensorflow:loss = 41584034000.0, step = 17301 (0.278 sec)
INFO:tensorflow:global_step/sec: 246.24
INFO:tensorflow:loss = 101104930000.0, step = 17401 (0.403 sec)
INFO:tensorflow:global_step/sec: 369.216
INFO:tensorflow:loss = 230161900000.0, step = 17501 (0.288

INFO:tensorflow:global_step/sec: 353.514
INFO:tensorflow:loss = 67850390000.0, step = 24501 (0.281 sec)
INFO:tensorflow:global_step/sec: 356.369
INFO:tensorflow:loss = 84590910000.0, step = 24601 (0.280 sec)
INFO:tensorflow:global_step/sec: 342.218
INFO:tensorflow:loss = 62043850000.0, step = 24701 (0.300 sec)
INFO:tensorflow:global_step/sec: 303.748
INFO:tensorflow:loss = 100316980000.0, step = 24801 (0.321 sec)
INFO:tensorflow:global_step/sec: 349.303
INFO:tensorflow:loss = 82092294000.0, step = 24901 (0.288 sec)
INFO:tensorflow:global_step/sec: 358.055
INFO:tensorflow:loss = 130877735000.0, step = 25001 (0.278 sec)
INFO:tensorflow:global_step/sec: 343.725
INFO:tensorflow:loss = 119025990000.0, step = 25101 (0.290 sec)
INFO:tensorflow:global_step/sec: 368.961
INFO:tensorflow:loss = 99303730000.0, step = 25201 (0.285 sec)
INFO:tensorflow:global_step/sec: 363.202
INFO:tensorflow:loss = 174341080000.0, step = 25301 (0.268 sec)
INFO:tensorflow:global_step/sec: 328.573
INFO:tensorflow:los

** Create a prediction input function and then use the .predict method off your estimator model to create a list or predictions on your test data. **

In [115]:
pred_input_func = tf.estimator.inputs.pandas_input_fn(
        x = X_test,
        batch_size = 10,
        num_epochs = 1,
        shuffle = False) 

In [116]:
predictions = dnn_model.predict(pred_input_func)

In [117]:
preds = list(predictions)

INFO:tensorflow:Restoring parameters from /tmp/tmpdo3byo3a/model.ckpt-26000


In [118]:
preds

[{'predictions': array([239859.25], dtype=float32)},
 {'predictions': array([254570.97], dtype=float32)},
 {'predictions': array([205609.38], dtype=float32)},
 {'predictions': array([127398.625], dtype=float32)},
 {'predictions': array([250474.78], dtype=float32)},
 {'predictions': array([220204.88], dtype=float32)},
 {'predictions': array([268484.94], dtype=float32)},
 {'predictions': array([176661.25], dtype=float32)},
 {'predictions': array([178870.44], dtype=float32)},
 {'predictions': array([58044.938], dtype=float32)},
 {'predictions': array([226780.84], dtype=float32)},
 {'predictions': array([226830.16], dtype=float32)},
 {'predictions': array([182190.03], dtype=float32)},
 {'predictions': array([137257.5], dtype=float32)},
 {'predictions': array([323414.72], dtype=float32)},
 {'predictions': array([120033.51], dtype=float32)},
 {'predictions': array([189268.53], dtype=float32)},
 {'predictions': array([168966.16], dtype=float32)},
 {'predictions': array([149129.97], dtype=floa

In [119]:
type(predictions)

generator

** Calculate the RMSE. You should be able to get around 100,000 RMSE (remember that this is in the same units as the label.) Do this manually or use [sklearn.metrics](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) **

In [120]:
type(y_test)

pandas.core.series.Series

In [121]:
y_test.describe

<bound method NDFrame.describe of 16086    406200.0
8816     500001.0
7175     122900.0
16714    209100.0
14491    500001.0
11807    135500.0
19109    194100.0
6926     158100.0
11649    279200.0
11961    193800.0
3859     279400.0
8128     199100.0
2335      78900.0
1829     186900.0
15832    355600.0
19748    128800.0
5562     159600.0
14477    277800.0
9178     333700.0
5268     500001.0
12624    152800.0
17825    257400.0
18005    303400.0
9982     146900.0
5816     217500.0
8585     500001.0
19437    135600.0
2405      51500.0
14818    125000.0
12015    135600.0
           ...   
10042    117100.0
9346     316300.0
9776     164600.0
8730     270700.0
1545     350000.0
20237    206600.0
9421     478600.0
3673     165200.0
15571    222600.0
3108      84500.0
1647     485400.0
14693    129300.0
15242    439900.0
3321      77800.0
1095     153100.0
1356     125000.0
12985    117100.0
8107     143100.0
13390    100800.0
17132    189000.0
3275      80700.0
1493     322300.0
19718     72

In [122]:
preds[0]

{'predictions': array([239859.25], dtype=float32)}

In [123]:
pred_list = []

for p in preds:
    pred_list.append(p['predictions'])

In [124]:
pred_list

[array([239859.25], dtype=float32),
 array([254570.97], dtype=float32),
 array([205609.38], dtype=float32),
 array([127398.625], dtype=float32),
 array([250474.78], dtype=float32),
 array([220204.88], dtype=float32),
 array([268484.94], dtype=float32),
 array([176661.25], dtype=float32),
 array([178870.44], dtype=float32),
 array([58044.938], dtype=float32),
 array([226780.84], dtype=float32),
 array([226830.16], dtype=float32),
 array([182190.03], dtype=float32),
 array([137257.5], dtype=float32),
 array([323414.72], dtype=float32),
 array([120033.51], dtype=float32),
 array([189268.53], dtype=float32),
 array([168966.16], dtype=float32),
 array([149129.97], dtype=float32),
 array([217789.12], dtype=float32),
 array([130229.305], dtype=float32),
 array([172488.62], dtype=float32),
 array([206298.52], dtype=float32),
 array([124985.59], dtype=float32),
 array([101719.836], dtype=float32),
 array([260689.06], dtype=float32),
 array([182842.47], dtype=float32),
 array([59362.656], dtype=

In [125]:
y_pred = pd.Series(pred_list) 


In [126]:
type(y_pred)

pandas.core.series.Series

In [127]:
y_pred.describe

<bound method NDFrame.describe of 0        [239859.25]
1        [254570.97]
2        [205609.38]
3       [127398.625]
4        [250474.78]
5        [220204.88]
6        [268484.94]
7        [176661.25]
8        [178870.44]
9        [58044.938]
10       [226780.84]
11       [226830.16]
12       [182190.03]
13        [137257.5]
14       [323414.72]
15       [120033.51]
16       [189268.53]
17       [168966.16]
18       [149129.97]
19       [217789.12]
20      [130229.305]
21       [172488.62]
22       [206298.52]
23       [124985.59]
24      [101719.836]
25       [260689.06]
26       [182842.47]
27       [59362.656]
28       [169430.75]
29       [161454.44]
            ...     
6162     [233349.97]
6163     [353934.38]
6164     [193893.42]
6165     [275613.88]
6166      [77458.53]
6167     [280519.44]
6168     [211270.66]
6169     [199706.16]
6170     [148060.88]
6171     [156413.31]
6172     [202407.19]
6173     [127206.88]
6174     [240406.44]
6175     [110553.18]
6176     [196742.75]


In [128]:
type(y_test)

pandas.core.series.Series

In [129]:
y_test.describe

<bound method NDFrame.describe of 16086    406200.0
8816     500001.0
7175     122900.0
16714    209100.0
14491    500001.0
11807    135500.0
19109    194100.0
6926     158100.0
11649    279200.0
11961    193800.0
3859     279400.0
8128     199100.0
2335      78900.0
1829     186900.0
15832    355600.0
19748    128800.0
5562     159600.0
14477    277800.0
9178     333700.0
5268     500001.0
12624    152800.0
17825    257400.0
18005    303400.0
9982     146900.0
5816     217500.0
8585     500001.0
19437    135600.0
2405      51500.0
14818    125000.0
12015    135600.0
           ...   
10042    117100.0
9346     316300.0
9776     164600.0
8730     270700.0
1545     350000.0
20237    206600.0
9421     478600.0
3673     165200.0
15571    222600.0
3108      84500.0
1647     485400.0
14693    129300.0
15242    439900.0
3321      77800.0
1095     153100.0
1356     125000.0
12985    117100.0
8107     143100.0
13390    100800.0
17132    189000.0
3275      80700.0
1493     322300.0
19718     72

In [130]:
y_pred.describe

<bound method NDFrame.describe of 0        [239859.25]
1        [254570.97]
2        [205609.38]
3       [127398.625]
4        [250474.78]
5        [220204.88]
6        [268484.94]
7        [176661.25]
8        [178870.44]
9        [58044.938]
10       [226780.84]
11       [226830.16]
12       [182190.03]
13        [137257.5]
14       [323414.72]
15       [120033.51]
16       [189268.53]
17       [168966.16]
18       [149129.97]
19       [217789.12]
20      [130229.305]
21       [172488.62]
22       [206298.52]
23       [124985.59]
24      [101719.836]
25       [260689.06]
26       [182842.47]
27       [59362.656]
28       [169430.75]
29       [161454.44]
            ...     
6162     [233349.97]
6163     [353934.38]
6164     [193893.42]
6165     [275613.88]
6166      [77458.53]
6167     [280519.44]
6168     [211270.66]
6169     [199706.16]
6170     [148060.88]
6171     [156413.31]
6172     [202407.19]
6173     [127206.88]
6174     [240406.44]
6175     [110553.18]
6176     [196742.75]


In [131]:
from sklearn.metrics import mean_squared_error

In [132]:
mean_squared_error(y_test, y_pred)**0.5

107038.72823070538

# Great Job!